In [1]:
from tqdm import tqdm
from matplotlib import pyplot as plt
import pandas as pd
import re
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [2]:
df = pd.read_csv("./market_data/processed.csv")
# Rename the column 'emails' to 'email'
df = df.rename(columns={'emails': 'email'})
df = df[['title', 'email']]
df.head(2)

,title,email
0,Anfa uitzendbureau,info@anfa.nl
1,Aditech Utrecht,info@aditech.nl


In [3]:
df.shape

(83, 2)

In [4]:
# Find rows with duplicate emails
duplicate_rows = df[df['email'].str.count(',') >= 1]

# Print the emails with duplicates
print("Emails with duplicates:")
print(duplicate_rows[['title', 'email']])

Emails with duplicates:
                               title                                   email
59  Top Secretaries Midden Nederland  info@topsecretaries.nl, info@adecco.nl


In [5]:
# Split the 'email' column into multiple rows with the same title
df_copy = duplicate_rows.assign(email=duplicate_rows['email'].str.split(','))

# Explode the DataFrame to create a new row for each email
df_copy = df_copy.explode('email').reset_index(drop=True)

# Display the modified DataFrame
print("\nModified DataFrame:")
print(df_copy[['title', 'email']])


Modified DataFrame:
                              title                   email
0  Top Secretaries Midden Nederland  info@topsecretaries.nl
1  Top Secretaries Midden Nederland          info@adecco.nl


In [6]:
# Remove rows with duplicate emails from the original DataFrame
df = df[~df.index.isin(duplicate_rows.index)]

# Merge the modified DataFrame back into the original DataFrame
df = pd.concat([df, df_copy], ignore_index=True)

# Display the updated DataFrame
print("\nUpdated DataFrame:")


Updated DataFrame:


In [7]:
df.shape

(84, 2)

In [8]:
# Create a DataFrame
test_data = {'title': ['Bolar', 'Dexan Goldelux', 'xGino', 'Tresto BV', 'Doudella Agency'],
        'email': ['info@bolar.co', 'ginggeli@hotmail.com', 'qingge43@gmail.com', 'Mohamed.doudellla@gmail.com', 'Info@doudella.com']}

test_df = pd.DataFrame(test_data)

# Save the DataFrame to a CSV file
test_df.to_csv('test_emails.csv', index=False)

print("CSV file created successfully.")

# Load CSV data into DataFrame
test_df = pd.read_csv('test_emails.csv')
test_df.head()

CSV file created successfully.


,title,email
0,Bolar,info@bolar.co
1,Dexan Goldelux,ginggeli@hotmail.com
2,xGino,qingge43@gmail.com
3,Tresto BV,Mohamed.doudellla@gmail.com
4,Doudella Agency,Info@doudella.com


In [9]:
df = pd.concat([df, test_df], ignore_index=True)
df.shape

(89, 2)

In [11]:
df.tail(10)

,title,email
79,Carrière Vormer,info@carrierevormer.nl
80,FBRC GGZ Werving,info@ggz-werving.nl
81,Uitbestedingscentrum,info@uitbestedingscentrum.nl
82,Top Secretaries Midden Nederland,info@topsecretaries.nl
83,Top Secretaries Midden Nederland,info@adecco.nl
84,Bolar,info@bolar.co
85,Dexan Goldelux,ginggeli@hotmail.com
86,xGino,qingge43@gmail.com
87,Tresto BV,Mohamed.doudellla@gmail.com
88,Doudella Agency,Info@doudella.com


In [12]:
# Save the DataFrame to a CSV file
df.to_csv('send_email.csv', index=False)

In [ ]:
# Extract the part before the '@' symbol from each email address
df['email_prefix'] = df['email'].str.split('@').str[0]

# Plot a histogram of the email prefixes
df['email_prefix'].hist()

# Display the plot
plt.show()

In [13]:
# Email configuration
username = "info@bolar.co"
password = "Utrecht54321"
hostname = "bolar.co"
port = 465  # SMTP Port for SSL/TLS
sender_email = username

In [14]:
def send_email(df, sender_email, username, password, hostname, port):

    progress_bar = tqdm(total=len(df), desc="Sending Emails", unit="email")
    # Iterate over rows and send emails
    for index, row in df.iterrows():
        recipient_email = row['email']
        title = row['title']
        
        # Email content
        subject = "Ontdek de kracht van gerichte marketing voor jouw lokale bedrijf!"
        body = f"""Beste {title},<br />
            <br />
            Klaar om jouw lokale bedrijf te laten groeien met gerichte marketing? Ontdek onze exclusieve e-maillijsten voor lokale bedrijven!<br />            
            
            <ul>
                <li>✔️ <strong>Impact</strong>: Bereik potentiële klanten met een impactvolle benadering.</li>
                <li>✔️ <strong>Omzetboost</strong>: Verhoog je omzet met een uitgebreide lijst van e-mailadressen.</li>
                <li>✔️ <strong>Optimaliseer</strong>: Bereik jouw doelgroep persoonlijk en relevant.</li>
                <li>✔️ <strong>Waardevolle contacten</strong>: Bespaar tijd en moeite met directe toegang tot waardevolle contacten.</li>
            </ul>
            <br />
            Bekijk ons <strong>exclusieve aanbod</strong> op <a href="http://www.bolar.co">www.bolar.co</a> en vraag vrijblijvend een gratis offerte aan. Til jouw marketing naar een hoger niveau en zie direct resultaat<br />
            <br />
            Met vriendelijke groet,<br />
            Het team van Bolar
            """

        # Construct email message
        message = MIMEMultipart()
        message['From'] = sender_email
        message['To'] = recipient_email
        message['Subject'] = subject
        message.attach(MIMEText(body, 'html'))

        try:
            # Create SMTP session with SSL/TLS
            with smtplib.SMTP_SSL(hostname, port) as server:
                server.login(username, password)
                server.sendmail(sender_email, recipient_email, message.as_string())
                print(f"{recipient_email} email sent successfully.")
        except Exception as e:
            print(f"{recipient_email} Failed to send: {e}")
        
        # Update the progress bar
        progress_bar.update(1)

    # Close the progress bar
    progress_bar.close()

send_email(df, sender_email, username, password, hostname, port)

Sending Emails:   1%|          | 1/89 [00:00<01:19,  1.10email/s]

info@anfa.nl email sent successfully.


Sending Emails:   2%|▏         | 2/89 [00:01<01:02,  1.40email/s]

info@aditech.nl email sent successfully.


Sending Emails:   3%|▎         | 3/89 [00:01<00:53,  1.61email/s]

...

Sending Emails:  99%|█████████▉| 88/89 [01:10<00:00,  1.45email/s]

Mohamed.doudellla@gmail.com email sent successfully.


Sending Emails: 100%|██████████| 89/89 [01:10<00:00,  1.26email/s]

Info@doudella.com email sent successfully.


: 